In [ ]:
#test call for the library
using RateDistortionDecisionMaking


card_x = 3 #cardinality x
card_ϕ = 2 #cardinality ϕ
β = 1 #inverse temperature
ε = 0.000001 #convergence critetion for BAiterations
maxiter = 5000 #maximum number of BA iterations

#set up uniform p(ϕ)
pϕ = ones(card_ϕ)/card_ϕ 

#initialize p(x) uniformly
px_init = ones(card_x)/card_x 


#cosine utility function (returns pre-computed utilities and maximum for each y)
function cosineutility(cardinality_x, cardinality_ϕ)
    #set up x ∈ (0,π/2)
    xvec = linspace(0,pi/2,cardinality_x)
    #set up ϕ ∈ (0,π/4)
    ϕvec = linspace(0,pi/4,cardinality_ϕ)

    #set up the utility function
    #utility is cosine, ϕ acts as a phase shift
    function U(x,ϕ)
        cos(x-ϕ)
    end

    #pre-compute utilities, find maxima
    U_pre, Umax = setuputilityarrays(xvec,ϕvec,U)
    
    return U_pre, Umax, xvec, ϕvec
end


#exemplary usage
U_pre, Umax, xvec, ϕvec = cosineutility(card_x, card_ϕ)
#pxgϕ,px = BAiterations(px_init, β, U_pre, Umax, pϕ, ε, maxiter)

#pxgϕ, px, I, Ha, Hago, EU, RDobj = BAiterations(px_init, β, U_pre, Umax, pϕ, ε, maxiter,compute_performance=true, 
#performance_as_dataframe=false, performance_per_iteration=false)

pxgϕ, px, perf = BAiterations(px_init, β, U_pre, pϕ, ε, maxiter,compute_performance=true, 
performance_as_dataframe=true, performance_per_iteration=false)

#U_pre,pxgϕ,px
#perf

In [ ]:
#convert solution

#These are potential candidates for tests - all of these versions should run

#transform result into dataframe
#px_df, pxgϕ_df = BAresult2DataFrame(px, pxgϕ, xvec, ϕvec, map(string,xvec), map(string,ϕvec))
px_df, pxgϕ_df = BAresult2DataFrame(px, pxgϕ, xvec, ϕvec)  
#px_df = boltzmannresult2DataFrame(px, xvec)
#px_df = BAmarginal2DataFrame(px, xvec, map(string,xvec))
#pxgϕ_df = BAconditional2DataFrame(pxgϕ, xvec, ϕvec)  


#visualizeBAconditional(pxgϕ_df, xvec, ϕvec)
#visualizeBAconditional(pxgϕ_df, xvec, ϕvec,alabel="action x", olabel="observation ϕ", legendlabel="p(x|ϕ)")



In [ ]:
#visualizeBAmarginal(px, xvec)

#Two-step call explicitly using DataFrames
#px_df, pxgϕ_df = BAresult2DataFrame(px, pxgϕ, xvec, ϕvec)
visualizeBAsolution(px_df, pxgϕ_df, xvec, ϕvec)
; #to suppress output of the returned values


#Two-step call with strings provided
#px_df, pxgϕ_df = BAresult2DataFrame(px, pxgϕ, xvec, ϕvec, map(string,xvec), map(string,ϕvec))
#visualizeBAsolution(px_df, pxgϕ_df, xvec, ϕvec, olabel="ϕ", alabel="x", legendlabel_marginal="p(x)", legendlabel_conditional="p(x|ϕ)")

#Simplified call using vector and matrix
#visualizeBAsolution(px, pxgϕ, xvec, ϕvec)

In [ ]:
#vector/matrix call with strings provided
as = ["x1","x2","x3"]
os = ["ϕ1","ϕ2"]
plot_marg, plot_cond = visualizeBAsolution(px, pxgϕ, xvec, ϕvec, as, os,
                                           wlabel="ϕ", alabel="x",
                                           legendlabel_marginal="p(x)", legendlabel_conditional="p(x|ϕ)");

In [ ]:
#example for storing the figure
#using Gadfly
#draw(SVG("Figures/conditional.svg", 10cm, 10cm), plot_cond)
#draw(SVGJS("Figures/conditionaljs.svg", 4inch, 3inch), plot_cond)  #this embeds the JavaScript (for panning zooming, etc) into the SVG

#Temperature sweep

In [ ]:
β_sweep = [0.1:30]
nβ = length(β_sweep)

I = zeros(nβ)
Ha = zeros(nβ)
Hago = zeros(nβ)
EU = zeros(nβ)
RDobj = zeros(nβ)

for i=1:nβ    
    p_xgϕ, p_x, I[i], Ha[i], Hago[i], EU[i], RDobj[i] = BAiterations(px_init, β_sweep[i], U_pre, pϕ, ε, maxiter,compute_performance=true)  
end

plt_perf_entropy, plt_perf_util, plt_rateutility = plotperformancemeasures(I, Ha, Hago, EU, RDobj, β_sweep);


'RU_obj' in the figure legend denotes the value of the rate-utility objective function $E_{p(a,o)}[U(a,o)]-\frac{1}{\beta}I(A;O)$.

The shaded region in the rate-utility function plots indicates the infeasible region - systems in this region are theoretically impossible.

In [ ]:
#uncomment to store plots

#using Gadfly

#plt_performance = vstack(plt_perf_entropy, plt_perf_util)
#draw(SVG("Figures/Tempsweep.svg", 8.5cm, 11cm), plt_performance)
#draw(SVG("Figures/RateUtility.svg", 8.5cm, 7cm), plt_rateutility)